In [8]:
!dvc get https://github.com/iterative/dataset-registry tutorials/versioning/data.zip

In [10]:
!dvc add data


To track the changes with git, run:

	git add data.dvc

To enable auto staging, run:

	dvc config core.autostage true


In [11]:
!git add data.dvc .gitignore

In [2]:
import numpy as np
import sys
import os

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras import applications
from tensorflow.keras.callbacks import CSVLogger
from tqdm.keras import TqdmCallback
from tensorflow import keras 
from PIL import Image
import time 

import mlflow
from pathlib import Path

In [2]:
experiment_id = mlflow.create_experiment(
    "cat-dog VGG classifier", 
    artifact_location=Path.cwd().joinpath("./mlruns/mlruns.db").as_uri(),
    tags={"version":"v1", "priority":"P1"}
)

In [3]:
mlflow.set_tracking_uri("sqlite:///mlruns/mlruns.db")

In [3]:
experiment = mlflow.set_experiment(experiment_name="cat-dog VGG classifier")

In [4]:
experiment

<Experiment: artifact_location='file:///c:/Users/Mahad/Documents/Python/MLOps/DL_codes/mlruns/mlruns.db', experiment_id='1', lifecycle_stage='active', name='cat-dog VGG classifier', tags={'priority': 'P1', 'version': 'v1'}>

In [5]:
path = os.getcwd()

In [6]:
img_width, img_height = 150, 150

In [7]:
top_model_weights_path = 'model.h5'
train_data_dir = os.path.join('data', 'train')
validation_data_dir = os.path.join('data', 'validation')
cats_train_path = os.path.join(path, train_data_dir, 'cats')
nb_train_samples = 2 * len([name for name in os.listdir(cats_train_path)
                            if os.path.isfile(
                                os.path.join(cats_train_path, name))])


In [8]:
batch_size = 4

In [9]:
datagen = ImageDataGenerator(rescale=1. / 255)

# build the VGG16 network
model = applications.VGG16(include_top=False, weights='imagenet')

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

val_generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

Found 1000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.


In [10]:
model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))
x = model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)

outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


In [11]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
vgg16 (Functional)           (None, None, None, 512)   14714688  
_________________________________________________________________
global_average_pooling2d (Gl (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 513       
Total params: 14,715,201
Trainable params: 513
Non-trainable params: 14,714,688
_________________________________________________________________


In [12]:
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = keras.optimizers.Adam()
# learning schedulre
epochs = 100

In [13]:
train_acc_metric = keras.metrics.Accuracy()
val_acc_metric = keras.metrics.Accuracy()

In [19]:
with mlflow.start_run(run_name="exp-1",experiment_id=1, description="A model to classify between cat and dog example") as run:

    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("objective", "BinaryCrossentropy")
    mlflow.log_param("epochs", epochs)
    
    for epoch in range(epochs):
        
        for step, (inputs, targets) in enumerate(train_generator):
            
            with tf.GradientTape() as tape:
                # Forward pass.
                predictions = model(inputs)
                # Compute the loss value for this batch.
                targets = targets.reshape(4,1)
                # targets = np.expand_dims(targets, axis=0)
                loss_value = loss_fn(targets, predictions)

            # Get gradients of loss wrt the *trainable* weights.
            gradients = tape.gradient(loss_value, model.trainable_weights)
            # Update the weights of the model.
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))
            
            train_acc_metric.update_state(targets.reshape(4,1), predictions)
            
            if step % 20 == 0:
                print(
                    f"Training loss at step {step}: {loss_value:.4f}"
                )
                
        train_acc = train_acc_metric.result()
        print(f"Training acc over epoch: {train_acc:.4f}")
            
        mlflow.log_param("training accuracy", train_acc)
        
        train_acc_metric.reset_states()
        
        for val_inputs, val_targets in val_generator:
            val_predictions = model(val_inputs, training=False)
            val_acc_metric.update_state(val_targets.reshape(4,1), val_predictions)
        val_acc = val_acc_metric.result()
        
        mlflow.log_param("validation accuracy", val_acc)
        val_acc_metric.reset_states()
        print(f"Validation acc: {val_acc:.4f}")

Training loss at step 0: 0.6177
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000
Training acc over epoch: 0.0000


KeyboardInterrupt: 

In [23]:
predictions.shape

TensorShape([4, 1])

In [28]:
targets.reshape(4,1)

(4, 1)

In [11]:
optimizer = 'sgd'
loss_fn = 'binary_crossentropy'

In [12]:
model.compile(optimizer=optimizer,
              loss=loss_fn, metrics=[keras.metrics.Accuracy(), keras.metrics.Precision(), keras.metrics.Recall()])

In [13]:
epochs = 300

In [45]:
model.save_weights(top_model_weights_path)

In [ ]:
!dvc add model.h5 metrics.csv

## Serving a registered model

In [ ]:
import mlflow.pyfunc

In [ ]:
model_name = "VGG cat-dog classifier"
stage = 'Staging'

In [ ]:
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{stage}"
)

In [20]:
test_img = next(val_generator)

In [34]:
img_name = "dogs\\dog.6.jpg"

In [35]:
test_img = np.array(Image.open(os.path.join(train_data_dir, img_name)))
test_img = np.expand_dims(test_img, axis=0)

In [37]:
test_img.shape

(1, 488, 499, 3)

In [38]:
model.predict(test_img)

WARNI [tensorflow] Model was constructed with shape (None, 4, 4, 512) for input KerasTensor(type_spec=TensorSpec(shape=(None, 4, 4, 512), dtype=tf.float32, name='flatten_input'), name='flatten_input', description="created by layer 'flatten_input'"), but it was called on an input with incompatible shape (None, 488, 499, 3).


ValueError: in user code:

    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1478 predict_function  *
        return step_function(self, iterator)
    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\keras\engine\training.py:1468 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1259 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2730 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    c:\Users\Mahad\AppData\Local\Programs\Python\Python37\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3417 _call_for_each_re